In [1]:
# Mapeo de (x,y) a (r,c)
import numpy as np
mars_map = np.load('mars_map.npy')
nr, nc = mars_map.shape


$r = n_r - round(\frac{y}{cscala})$ 

$c = round(\frac{x}{cscala})$ 

Donde $n_r$ y $n_c$ son los números de renglones y columnas de la imagen o matriz de datos (en este caso 1814 y 756), y la escala indica la cantidad de metros por cada pixel (10.0174).

In [ ]:
# TODO: Cambiar variables (x,y) y no se si round este bien
cscala = 10.0174
r = nr - round('y'/cscala)
c =round('x'/cscala)